In [179]:
import numpy as np
import json
from geo_boundaries import *
from county_geo import *
from sklearn.externals import joblib
from sklearn.neighbors import KDTree
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import cnfg
import json
from geo import *
# from vaderSentiment.vaderSentiment import sentiment as vaderSentiment
# from textblob import TextBlob
from pymongo import MongoClient
import time
import logging
import logging.handlers
import datetime
import os.path
from bson.json_util import dumps
from __future__ import division
import math

In [119]:
# client = MongoClient()
# tweets = client.fletcher.tweets
# backup = tweets.find({"$text":{"$search":"\"hillary clinton\", hillary, clinton"}})
# hillary = tweets.aggregate([{"$match":{"$text":{"$search":"hillary"}}}])
# ,
#      {"$match":{"timestamp_ms":{"$gte":100000}}}])
# all_tweets = tweets.find()

In [120]:
# all_tweets = tweets.find()
# backup.count()
# query = tweets.aggregate([
#             {"$match":{"$text":{"$search":"hillary"}}}])

In [104]:
# count = 0
# for tweet in query:
#     count += 1

# print count
from __future__ import division
import flask
from pymongo import MongoClient
import time
from county_geo import *
from twitter_functions import *
import math
from datetime import date

def return_tweets(day, search_terms="", all_results=0):
    today_epoch = int(date.today().strftime('%s'))
    start_time = (today_epoch - (day*86400))*1000
    end_time = ((start_time + 86400)*1000)-1
    if all_results==1:
        start_time = 0
        end_time = (today_epoch+86400)*1000
    client = MongoClient()
    db = client.fletcher.tweets
    tweets = {}
    if search_terms != "":
        query = db.aggregate([
            {"$match":{"$text":{"$search":search_terms}}},
            {"$match":{"timestamp_ms":{"$gte":start_time,"$lt":end_time}}}])
    else:
        query = db.aggregate([
            {"$match":{"timestamp_ms":{"$gte":start_time,"$lt":end_time}}}])
    for tweet in query:
        fips = tweet['fips']
        if fips in tweets:
            tweets[fips]['volume'] += 1
            try:
                tweets[fips]['sentiment'] += tweet['sentiment']['compound']
            except:
                pass
        else:
            tweets[fips] = {}
            tweets[fips]['volume'] = 1
            try:
                tweets[fips]['sentiment'] = tweet['sentiment']['compound']
            except:
                tweets[fips]['sentiment'] = 0
    for x in tweets:
        tweets[x]['sentiment'] = tweets[x]['sentiment']/tweets[x]['volume']

    return tweets

def tweet_booststrapper(dict, n=0):
    bootstrap_dict = {}
    for key in all_fips:
        try:
            volume = dict[key]['volume']
            sentiment = dict[key]['sentiment']
        except:
            volume = 0
            sentiment = None
        if volume < 100:
            new_sent, new_vol, weight, new_mult = 0, 0, 10, 0
            for county in nearest_counties[key]:
                try:
                    vol = dict[county]['volume']*weight
                    new_vol += vol
                    new_sent += dict[county]['sentiment']*multiplier
                    new_mult += multiplier
                except:
                    pass
                weight -= 1
            new_vol = (volume*(volume/100))+((new_vol/55)*((100-volume)/100))
            if new_mult > 0:
                new_sent = (volume*(volume/100))+((new_sent/new_mult)*((100-volume)/100))
            else:
                new_sent = sentiment
        else:
            new_vol = volume
            new_sent = sentiment
        bootstrap_dict[key] = {}
        bootstrap_dict[key]['volume'] = new_vol
        bootstrap_dict[key]['sentiment'] = new_sent
    if n > 0:
        n -= 1
        return tweet_booststrapper(bootstrap_dict, n)
    return bootstrap_dict


def get_all_candidates(time=0, n=0, all_results=0):
    candidate_search = { "Lincoln Chafee" : "lincolnchafee chafee teamchafee teamlincolnchafee chafee2016 lincolnchafee2016 chafeeforpresident chafee4president",  "Hillary Clinton" : "hillaryclinton hillary teamhillary teamclinton teamhillaryclinton hillary2016 clinton2016 hillaryclinton2016 hillaryforamerica hillary4america hillaryforpresident hillary4president",  "Martin O'Malley" : "governoromalley martinomalley teamomalley teammartinomalley omalley2016 martinomalley2016 omalley4president omalleyforpresident",  "Bernie Sanders" : "berniesanders sensanders teambernie teamberniesanders bernie2016 sanders2016 berniesanders2016 sanders4president sandersforpresident bernie4president bernieforpresident berniesanderse4president berniesanderseforpresident",  "Ben Carson" : "realbencarson bencarson drbencarson teamcarson teambencarson bencarson2016 carson2016 carson4president carsonforpresident bencarson4president bencarsonforpresident",  "Ted Cruz" : "tedcruz sentedcruz teamcruz teamtedcruz tedcruz2016 cruzcrew cruz4president cruzforpresident tedcruz4president tedcruzforpresident",  "Carly Fiorina" : "carlyfiorina fiorina teamcarly teamfiorina teamcarlyfiorina carly2016 carly4president carlyforpresident carlyfiorina4president carlyfiorinaforpresident fiorina4president fiorinaforpresident",  "Lindsey Graham" : "linseygrahamsc lindseygraham grahamblog senlindseygraham teamgraham teamlindseygraham graham2016 lindseygraham2016 graham4president grahamforpresident lindseygraham4president lindseygrahamforpresident",  "George Pataki" : "governorpataki govpataki pataki georgepataki pataki4president patakiforpresident georgepataki4president georgepatakiforpresident teampataki",  "Rand Paul" : "randpaul drrandpaul senatorrandpaul teamrand teamrandpaul randpaul2016 rand2016 standwithrand rand4president randforpresident paul4president paulforpresident randpaul4president randpaulforpresident",  "Rick Perry" : "governorperry rickperry teamrickperry perry2016 rickperry2016 perry4president perryforpresident rickperry4president rickperryforpresident",  "Marco Rubio" : "marcorubio rubio2016 marcorubio2016 teammarco teammarcorubio marco4president marcoforpresident rubio4president rubioforpresident marcorubio4president marcorubioforpresident",  "Rick Santorum" : "ricksantorum santorum2016 teamsantorum teamricksantorum santorum4president santorumforpresident ricksantorum4president ricksantorumforpresident",  "Mike Huckabee" : "govmikehuckabee mikehuckabee teamhuckabee teammikehuckabee huckabee2016 mikehuckabee2016 huckaboom huckabee4president huckabeeforpresident mikehuckabee4president mikehuckabeeforpresident",  "Jeb Bush" : "jebbush  teamjeb teamjebbush jeb2016 bush2016 jebbush2016 jeb jeb4president jebforpresident jebbush4president jebbushforpresident",  "Scott Walker" : "scottwalker teamscottwalker scottwalker2016 walker4president walkerforpresident scottwalker4president scottwalkerforpresident" }
    candidate_tweets = {}
    candidate_totals = {}
    for key in candidate_search:
        total_volume = 0
        total_sentiment = 0
        total_sent_vol = 0
        search_terms = candidate_search[key]
        tweets = return_tweets(time, search_terms, all_results)
        boosted_tweets = tweet_booststrapper(tweets,n)
        candidate_tweets[key] = boosted_tweets
        for county in boosted_tweets:
            county_volume = boosted_tweets[county]['volume']
            county_sentiment = boosted_tweets[county]['sentiment']
            total_volume += county_volume
            if county_sentiment != None:
                total_sentiment += (county_volume*county_sentiment)
                total_sent_vol += county_volume
        candidate_totals[key] = {}
        try:
            candidate_totals[key]['volume'] = total_volume
            candidate_totals[key]['sentiment'] = total_sentiment/total_sent_vol
        except:
            candidate_totals[key]['volume'] = 0
            candidate_totals[key]['sentiment'] = 0
    all_data = {"total" : candidate_totals, "county" : candidate_tweets}
    return all_data

In [105]:
# all_candidates = get_all_candidates(1000)
tweets = tweet_booststrapper(return_tweets(1))
# tweets = return_tweets(1)

In [106]:
tweets
# all_candidates

{54065: {'sentiment': 0.0, 'volume': 24.166},
 54011: {'sentiment': 0.1970754098360656, 'volume': 40.28036363636364},
 42009: {'sentiment': 0.06268333333333334, 'volume': 8.888363636363636},
 46101: {'sentiment': None, 'volume': 10.090909090909092},
 30007: {'sentiment': 0.4017, 'volume': 11.62},
 30043: {'sentiment': None, 'volume': 8.254545454545454},
 9003: {'sentiment': 0.20673880208333317, 'volume': 384},
 41003: {'sentiment': 0.10554468085106383, 'volume': 50.52690909090909},
 41005: {'sentiment': 0.1036911504424779, 'volume': 113},
 41007: {'sentiment': 0.42863684210526326, 'volume': 34.787636363636366},
 41009: {'sentiment': 0.35995, 'volume': 172.24436363636363},
 41011: {'sentiment': 0.17868181818181816, 'volume': 30.01363636363637},
 41013: {'sentiment': None, 'volume': 8.545454545454545},
 41015: {'sentiment': 0.10296666666666666, 'volume': 24.69272727272727},
 41017: {'sentiment': 0.16212857142857146, 'volume': 24.568363636363635},
 41019: {'sentiment': -0.0141521739130434

In [168]:
# all_candidates
bs_tweets = tweet_booststrapper(tweets)

In [169]:
bs_tweets

{54065: {'sentiment': 0.08868292682926829, 'volume': 105.80909090909091},
 54011: {'sentiment': 0.0992646351242983, 'volume': 242.92727272727274},
 42009: {'sentiment': 0.08742803347280335, 'volume': 81.27272727272728},
 46101: {'sentiment': 0.0796736842105263, 'volume': 42.236363636363635},
 30007: {'sentiment': 0.0, 'volume': 36.627272727272725},
 30043: {'sentiment': 0.022288235294117647, 'volume': 28.87272727272727},
 9003: {'sentiment': 0.14354445747800584, 'volume': 1922.3545454545456},
 41003: {'sentiment': 0.13749286498353455, 'volume': 361.72727272727275},
 41005: {'sentiment': 0.16612479414455616, 'volume': 1600.1909090909091},
 41007: {'sentiment': 0.10159348795718111, 'volume': 317.3818181818182},
 41009: {'sentiment': 0.08471231527093599, 'volume': 705.6818181818181},
 41011: {'sentiment': 0.0997190082644628, 'volume': 165.39090909090908},
 41013: {'sentiment': 0.13577419354838707, 'volume': 52.972727272727276},
 41015: {'sentiment': 0.16174875000000002, 'volume': 103.0909

In [11]:
fips = {}

for num in all_fips:
    fips[num] = [0,0]

for doc in all_tweets:
    fips_num = doc['fips']
    try:
        fips_sent = doc['sentiment']['compound']
    except:
        fips_sent = 0
    if fips_num in fips:
        fips[fips_num][0] += 1
        fips[fips_num][1] += fips_sent
        
for key in fips:
    if fips[key][0] != 0:
        fips[key][1] = float(fips[key][1]/fips[key][0])

sentiment = {}
tweets_total = {}
tweet_density = {}

for key in fips:
    sentiment[key] = fips[key][1]
    tweets_total[key] = fips[key][0]
    tweet_density[key] = (fips[key][0]/fips_pop[key])*1000

In [20]:
no_tweets = []
for k, v in tweets_total.iteritems():
    if v == 0:
        no_tweets.append(k)

In [22]:
# tweets_total

In [36]:
# nearest_counties_dictionary = {}

# for key in geo_county_dictionary:
#     nearest_counties = find_county(geo_county_dictionary[key], neighbors=11, all=1)
#     nearest_counties_dictionary[key] = nearest_counties[1:]
    
# json.dump(nearest_counties_dictionary, open("nearest_counties_dictionary.py",'w'))

def county_booster(fips_dict, other_dict):
    boosted_dict = {}
    for key in fips_dict:
        total_vol = fips_dict[key]
        new_score = total_vol*other_dict[key]*15
        total = (15*total_vol)
        if total_vol < 10:
            index = 10
            for county in nearest_counties[key]:
                new_score += (other_dict[county]*index)
                index -= 1
        new_score = new_score / (total+55)
        boosted_dict[key] = new_score
    return boosted_dict

In [31]:
# county_booster(tweets_total, sentiment)

0.110476612392


{41045: 0.11047661239168362}

In [37]:
boosted_sentiment = county_booster(tweets_total, sentiment)
boosted_density = county_booster(tweets_total, tweet_density)

In [38]:
with open('boosted_sentiment.tsv', 'w') as f:
    [f.write('{0}\t{1}\n'.format(key, value)) for key, value in boosted_sentiment.items()]

In [39]:
with open('tweets.tsv', 'w') as f:
    [f.write('{0}\t{1}\n'.format(key, value)) for key, value in tweets_total.items()]

In [40]:
with open('boosted_tweet_density.tsv', 'w') as f:
    [f.write('{0}\t{1}\n'.format(key, value)) for key, value in boosted_density.items()]

In [62]:
# new_dict = {}
# for key in all_county_boundaries:
#     points = all_county_boundaries[key]
#     nest_level = find_nested(points)
#     if nest_level == 0:
#         new_array = []
#         for point in points:
#             new_array.append([point[1],point[0]])
#         new_dict[key] = new_array
#     else:
#         new_array = []
#         for sub_points in points:
#             sub_array = []
#             for point in sub_points:
#                 sub_array.append([point[1],point[0]])
#             new_array.append(sub_array)
#         new_dict[key] = new_array

In [67]:
# f = open('new_file.py', 'w')
# f.write(str(new_dict)) 
# f.close()

In [7]:
# county_geo_dictionary[40.776557, -73.970174]

In [96]:
def in_polygon(point, poly):
  x = point[0]
  y = point[1]
  n = len(poly)
  status = False
  p1x,p1y = poly[0]
  for i in range(n+1):
    p2x,p2y = poly[i % n]
    if min(p1y,p2y) < y <= max(p1y,p2y):
      if x <= max(p1x,p2x):
        if p1y != p2y:
          xinters = (y-p1y)*(p2x-p1x)/(p2y-p1y)+p1x
        if p1x == p2x or x <= xinters:
          status = not status
    p1x,p1y = p2x,p2y

  return status

def find_nested(nested_list):
    try:
        nested_list[0][0][0]
        nest = 1
    except:
        nest = 0
    return nest
    
def find_county(point):
    county_tree = joblib.load('geo_tree/county_tree.pkl') 
    dist, indices = county_tree.query(point, k=5)
    fips_list = []
    for index in indices[0]:
        try:
            fips = county_geo_dictionary[(county_geo[index][0],county_geo[index][1])]
            fips_list.append(fips)
        except:
            pass
    if len(fips_list) > 0:
        try:
            for fips_num in fips_list:
                try:
                    county_paths = all_county_boundaries[str(fips_num)]
                    if find_nested(county_paths) == 0:
                        if in_polygon(point, county_paths):
                            return fips_num
                            break
                    else:
                        for path in county_paths:
                            try:
                                if in_polygon(point, path):
                                    return fips_num
                                    break
                            except:
                                pass
                except:
                    pass
        except:
            return fips_list[0]
    else:
        return None
    if dist[0][0] > 6:
        return None
    return fips_list[0]

In [97]:
print find_county([40.533548, -97.290977])

31151


In [14]:
locations = [["bronx", [[40.915132, -73.912168],[40.799059, -73.918005],[40.814391, -73.779302],[40.881655, -73.782735]]], ["harlem", [[40.834370, -73.950320],[40.818913, -73.961478],[40.801243, -73.959247],[40.797084, -73.949290],[40.817614, -73.934012],[40.828006, -73.934871]]], ["upper_west", [[40.818913, -73.961478],[40.801243, -73.959247],[40.768767, -73.982015],[40.773057, -73.993516]]], ["upper_east", [[40.797084, -73.949290],[40.817614, -73.934012],[40.764275, -73.972852],[40.758798, -73.958775],[40.776219, -73.941094],[40.799159, -73.928306],]], ["midtown", [[40.801243, -73.959247],[40.768767, -73.982015],[40.764275, -73.972852],[40.758798, -73.958775],[40.726881, -73.971736],[40.745416, -74.015509]]], ["downtown", [[40.726881, -73.971736],[40.745416, -74.015509],[40.697279, -74.021947],[40.711984, -73.971307]]], ["brooklyn", [[40.738697, -73.965254],[40.707995, -73.972120],[40.704351, -73.996839],[40.674674, -74.029112],[40.607982, -74.049024],[40.559485, -74.020185],[40.601727, -73.820371],[40.694981, -73.869810]]], ["queens", [[40.694981, -73.869810],[40.738697, -73.965254],[40.791007, -73.908958],[40.801923, -73.775748],[40.731196, -73.705710],[40.574394, -73.754462],[0.544137, -73.952216],[40.566049, -73.915824],[40.601727, -73.820371]]], ["san_fran", [[37.707196, -122.503175],[37.711270, -122.348679],[37.837188, -122.364366],[37.800303, -122.527787]]], ["west_bay", [[37.416526, -121.994411],[37.361970, -122.120067],[37.627261, -122.498996],[37.707196, -122.503175],[37.711270, -122.348679]]], ["east_bay", [[37.486844, -121.872875],[37.451966, -121.947033],[37.502835, -122.102708],[37.961171, -122.443285],[38.080176, -122.165880]]], ["south_bay", [[37.486844, -121.872875],[37.451966, -121.947033],[37.416526, -121.994411],[37.361970, -122.120067],[37.222672, -121.927807],[37.305189, -121.750652]]]]
def find_location(point):
  for area in locations:
    if in_polygon(point,area[1]):
      return area[0]
      break
  else:
    return None

print find_location([40.749898, -73.985726])

midtown
